# Modified Gale Shapley Algorithm for Generative Data Comparison

### Init

In [51]:
import pandas as pd

In [52]:
syndata = pd.read_csv('synData9.csv')
realdata = pd.read_csv('kag_risk_factors_cervical_cancer.csv')

In [53]:
syndata.iloc[0] #Use this to access a row
syndata.loc[0, :] #Use this to access a row
x = syndata.loc[:,"Age"] #Use this to access a column
syndata.columns[0]

'Age'

### Data Cleaning and Helper Functions

Code for finding the mean which ignores all the unknown values.

In [54]:
def mean_for_column(data1, column):
    """
    Finds the mean for a specific column
    """
    tot = 0 
    for i in range(len(data1)):
        #print(data1.loc[i, column])
        if data1.loc[i, column] == "?" or data1.loc[i, column] == -1:
            pass
        else:
            tot = tot + float(data1.loc[i, column])
    tot = tot/(len(data1))
    return tot
    
def mean_for_cols(data1):
    """
    Computes the mean for all the columns 
    """
    ar = []
    for col in data1.columns: 
        ar.append(mean_for_column(data1, col))
    return ar

In [55]:
syndata_means = mean_for_cols(syndata)
realdata_means = mean_for_cols(realdata)

clean(data1) cleans the code by removing any missing values and making them into -1

In [56]:
def clean(data1):
    """
    Replaces the missing data with a -1 value and converts the strings into floats
    """
    for i in range(len(data1)):
        for f in data1.columns:
            #print(data1.loc[i, f])
            if data1.loc[i, f] == "?":
                data1.loc[i, f] = float(-1)
                continue
            if isinstance(data1.loc[i, f], str):
                data1.loc[i, f] = float(data1.loc[i, f])
    #data1 =data1.set_axis([f for f in range(len(data1))], axis='index')
    #print(data1.dtypes)
    return data1.astype('float')

In [57]:
#Back up of the data
syndata_safe = syndata 
realdata_safe = realdata

realdata_clean = clean(realdata)
syndata_clean = clean(syndata)

realdata = realdata_safe
syndata = syndata_safe

Normalization function

In [58]:
def norm(data1):
    """
    Min-Max normalizes the data
    """
    new_dat = data1.copy()
    for col in new_dat.columns:
        new_dat[col] = (new_dat[col] - new_dat[col].min()) / (new_dat[col].max() - new_dat[col].min())
    return new_dat

In [59]:
#For syndata
syndata_clean_norm = norm(syndata_clean)
#For realdata
realdata_clean_norm = norm(realdata_clean)

### Weights

In [60]:
#We start of by getting the column names
#print(realdata_clean_norm.columns)
#print(syndata_clean_norm.columns)

In [61]:
#They're the same so we use 1 single column array
cols = ['Age', 'Number of sexual partners', 'First sexual intercourse',
       'Num of pregnancies', 'Smokes', 'Smokes (years)', 'Smokes (packs/year)',
       'Hormonal Contraceptives', 'Hormonal Contraceptives (years)', 'IUD',
       'IUD (years)', 'STDs', 'STDs (number)', 'STDs:condylomatosis',
       'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
       'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
       'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
       'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
       'STDs:Hepatitis B', 'STDs:HPV', 'STDs: Number of diagnosis',
       'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis',
       'Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
       'Citology', 'Biopsy']

#Init
real_weights =[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
syn_weights =[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]


high_weights =['Age', 'Number of sexual partners', 'First sexual intercourse',
       'Num of pregnancies', 'Smokes']
med_weights =['Smokes (years)', 'Smokes (packs/year)',
       'Hormonal Contraceptives', 'Hormonal Contraceptives (years)', 'IUD',
       'IUD (years)', 'STDs', 'STDs (number)', 'STDs:condylomatosis',
       'STDs:cervical condylomatosis']
low_weights =['STDs:vaginal condylomatosis',
       'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
       'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
       'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
       'STDs:Hepatitis B', 'STDs:HPV', 'STDs: Number of diagnosis',
       'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis',
       'Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
       'Citology', 'Biopsy']

for i in range(len(cols)):
    if cols[i] in high_weights:
        real_weights[i] = 3
        syn_weights[i] = 3
    if cols[i] in med_weights:
        real_weights[i] = 2
        syn_weights[i] = 2
    if cols[i] in low_weights:
        real_weights[i] = 1
        syn_weights[i] = 1

### Comparison function

Now we work on creating a compare function which is being designed to comapre all the rows of one dataset to one specific row of another dataset.

In [62]:
#lst = []
#pd.DataFrame(lst)  

def compare(data1, data2, row_comp, col, weights):
    """
    This is a comparsion function  
    data1: First dataset for comparsion, 1 set row
    data2: Second dataset for comparsion
    row_comp: The row we want compared
    """
    #Goes for 1 section of the real data and then loops for every row of the synthesized data
    lst = []
    for i in range(len(data2)):
        print('Real data: ' + str(data1.loc[row_comp, col]) + '. Fake data: ' +str(data2.loc[i, 'Age']))
        #Comparsion function to compare each the set real row to current fake row
        lst.append(similiar(data1, data2, i, row_comp, weights))
        """
        if i==1:
            return
        """
    ret = min(lst)
    return ret

In [63]:
def similiar(data1, data2, ind, row_comp, weights): #, weight):
    """
    Computes how similiar the two datasets are 
    """
    score = 0
    for i in data1.columns:
        score = score + ((data1.loc[row_comp,:] - data2.loc[ind,:]) * weights[i])
    return score 
    """
    for i in data1.columns:
        print(ind, data1.loc[row_comp,:], data2.loc[ind,:])
        print(i)
        if i ==1:
            return
    """

In [64]:
compare(realdata_clean_norm,syndata_clean_norm, 0, "Age", real_weights)

Real data: 0.07042253521126761. Fake data: 0.03636363636363636


TypeError: list indices must be integers or slices, not str

In [ ]:
realdata_clean_norm.dtypes

Age                                   float64
Number of sexual partners              object
First sexual intercourse               object
Num of pregnancies                     object
Smokes                                 object
Smokes (years)                         object
Smokes (packs/year)                    object
Hormonal Contraceptives                object
Hormonal Contraceptives (years)        object
IUD                                    object
IUD (years)                            object
STDs                                   object
STDs (number)                          object
STDs:condylomatosis                    object
STDs:cervical condylomatosis           object
STDs:vaginal condylomatosis            object
STDs:vulvo-perineal condylomatosis     object
STDs:syphilis                          object
STDs:pelvic inflammatory disease       object
STDs:genital herpes                    object
STDs:molluscum contagiosum             object
STDs:AIDS                         